# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

<b>Шаги для выбора локации:</b>

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<b>Инструкция по выполнению проекта</b>

- <b>1. Загрузите и подготовьте данные. Поясните порядок действий.</b>
- <b>2. Обучите и проверьте модель для каждого региона:</b>
- 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
- 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
- 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- 2.5. Проанализируйте результаты.
<br>
<b>3. Подготовьтесь к расчёту прибыли:</b>
- 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
- 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- 3.3. Напишите выводы по этапу подготовки расчёта прибыли.
<br>
<b>4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:</b>
- 4.1. Выберите скважины с максимальными значениями предсказаний.
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.
<br>
<b>5. Посчитайте риски и прибыль для каждого региона:</b>
- 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

## Загрузка и подготовка данных

In [1]:
#подключим необходимые для работы библиотеки

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.3f}'.format

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.preprocessing import StandardScaler

In [2]:
#прочитаем исходные датафреймы

try:
    geo_data_0 = pd.read_csv('C:/Users/jnybfuvybtfgl kfolbl/mypythonworks/yandex/geo_data_0.csv', sep=',') 
    geo_data_1 = pd.read_csv('C:/Users/jnybfuvybtfgl kfolbl/mypythonworks/yandex/geo_data_1.csv', sep=',') 
    geo_data_2 = pd.read_csv('C:/Users/jnybfuvybtfgl kfolbl/mypythonworks/yandex/geo_data_2.csv', sep=',') 
except:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv', sep=',')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv', sep=',')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv', sep=',')

In [3]:
#изучим основную информацию об исходных датафреймах

dataframes = [geo_data_0, geo_data_1, geo_data_2]
names = ['geo_data_0', 'geo_data_1', 'geo_data_2']

count = -1
for name in names:
    count += 1
    display(name)
    print('Первые 2 строки датафрейма:')
    display(dataframes[count].head(2))
    print('Основная информация о датафрейме:')
    display(dataframes[count].info())
    print('Доля пропущенных данных в датафрейме:')
    display(dataframes[count].isna().mean())
    print('Количество дубликатов: ')
    display(dataframes[count].duplicated().sum())

'geo_data_0'

Первые 2 строки датафрейма:


,id,f0,f1,f2,product
0,txEyH,0.706,-0.498,1.221,105.280
1,2acmU,1.335,-0.340,4.365,73.038


Основная информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

...

None

Доля пропущенных данных в датафрейме:


id        0.000
f0        0.000
f1        0.000
f2        0.000
product   0.000
dtype: float64

Количество дубликатов: 


0

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Прежде чем перейти к обучению и проверке модели удалим столбец 'id' из каждого датафрейма, т.к. он нам не пригодится в обучении.
</div>

In [4]:
#удаляем столбец 'id'

geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Изучив исходные данные трех датафреймов можно сказать, что данные представлены в полном объеме, типы данных корректны. Также в представленных датафреймах уже имеется целевой признак - "product". 
</div>

## Обучение и проверка модели

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Так как у нас три датафрейма, напишем функцию для разделения датафреймов на обучающую и валидационную выборки.
</div>

In [5]:
def test_split(data, target):
    features = data.drop([target], axis=1)
    target = data[target]
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    return features_train, features_valid, target_train, target_valid   

In [6]:
# разделим датафрейм 'geo_data_0' на обучающую и валидационную выборки

features_train_0, features_valid_0, target_train_0, target_valid_0 = test_split(geo_data_0, 'product')
print('Размер обучающей выборки равен: ', features_valid_0.shape)
print('Размер валидационной выборки равен: ', features_train_0.shape)

Размер обучающей выборки равен:  (25000, 3)
Размер валидационной выборки равен:  (75000, 3)


In [7]:
# разделим датафрейм 'geo_data_1' на обучающую и валидационную выборки

features_train_1, features_valid_1, target_train_1, target_valid_1 = test_split(geo_data_1, 'product')
print('Размер обучающей выборки равен: ', features_valid_1.shape)
print('Размер валидационной выборки равен: ', features_train_1.shape)

Размер обучающей выборки равен:  (25000, 3)
Размер валидационной выборки равен:  (75000, 3)


In [8]:
# разделим датафрейм 'geo_data_2' на обучающую и валидационную выборки

features_train_2, features_valid_2, target_train_2, target_valid_2 = test_split(geo_data_2, 'product')
print('Размер обучающей выборки равен: ', features_valid_2.shape)
print('Размер валидационной выборки равен: ', features_train_2.shape)

Размер обучающей выборки равен:  (25000, 3)
Размер валидационной выборки равен:  (75000, 3)


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее проведем масштабирование признаков с помощью их стандартизации, кроме целевого признака 'product'.
</div>

In [9]:
#напишем функцию для масштабирования

def scale(features_train, features_valid):

    numeric = ['f0', 'f1', 'f2']

    scaler = StandardScaler()
    scaler.fit(features_train[numeric]) 

    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric]= scaler.transform(features_valid[numeric]) 
    
    return features_train, features_valid

In [10]:
#применим функцию к выборкам с признаками

features_train_0_scale, features_valid_0_scale = scale(features_train_0, features_valid_0)

features_train_1_scale, features_valid_1_scale = scale(features_train_1, features_valid_1)

features_train_2_scale, features_valid_2_scale = scale(features_train_2, features_valid_2)

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Теперь можно перейти к обучению модели линейной регрессии.
</div>

In [11]:
model = LinearRegression()

def predict(features_train, target_train, features_valid, target_valid):
    
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    data = {'init_product': target_valid,
            'predicted_product': predicted_valid}
    
    dataframe = pd.DataFrame(data, columns = ['init_product', 'predicted_product'])
    mean_predicted_product = dataframe['predicted_product'].mean()
    
    return dataframe, rmse, mean_predicted_product

In [12]:
#применим функцию к выборкам с признаками и целевым признаком для датафрейма "geo_data_0"

geo_data_0_predicted, geo_data_0_rmse, geo_data_0_predicted_mean = predict(
    features_train_0_scale, target_train_0, features_valid_0_scale, target_valid_0)

print(geo_data_0_predicted.head())
print("Средний запас предсказанного сырья в первом регионе, тыс.баррелей: {:.3f}".format(geo_data_0_predicted_mean))
print("Средний запас начального сырья в первом регионе, тыс.баррелей: {:.3f}".format((geo_data_0_predicted['init_product'].mean())))
print("RMSE модели: {:.3f}".format(geo_data_0_rmse))

       init_product  predicted_product
71751        10.039             95.895
80493       114.551             77.573
2655        132.604             77.893
53233       169.072             90.175
91141       122.325             70.510
Средний запас предсказанного сырья в первом регионе, тыс.баррелей: 92.593
Средний запас начального сырья в первом регионе, тыс.баррелей: 92.079
RMSE модели: 37.579


In [13]:
#применим функцию к выборкам с признаками и целевым признаком для датафрейма "geo_data_1"

geo_data_1_predicted, geo_data_1_rmse, geo_data_1_predicted_mean = predict(
    features_train_1_scale, target_train_1, features_valid_1_scale, target_valid_1)

print(geo_data_1_predicted.head())
print("Средний запас предсказанного сырья во втором регионе, тыс.баррелей: {:.3f}".format(geo_data_1_predicted_mean))
print("Средний запас начального сырья во втором регионе, тыс.баррелей: {:.3f}".format((geo_data_1_predicted['init_product'].mean())))
print("RMSE модели: {:.3f}".format(geo_data_1_rmse))

       init_product  predicted_product
71751        80.860             82.663
80493        53.907             54.432
2655         30.132             29.749
53233        53.907             53.552
91141         0.000              1.244
Средний запас предсказанного сырья во втором регионе, тыс.баррелей: 68.729
Средний запас начального сырья во втором регионе, тыс.баррелей: 68.723
RMSE модели: 0.893


In [14]:
#применим функцию к выборкам с признаками и целевым признаком для датафрейма "geo_data_2"

geo_data_2_predicted, geo_data_2_rmse, geo_data_2_predicted_mean = predict(
    features_train_2_scale, target_train_2, features_valid_2_scale, target_valid_2)

print(geo_data_2_predicted.head())
print("Средний запас предсказанного сырья в третьем регионе, тыс.баррелей: {:.3f}".format(geo_data_2_predicted_mean))
print("Средний запас начального сырья в третьем регионе, тыс.баррелей: {:.3f}".format((geo_data_2_predicted['init_product'].mean())))
print("RMSE модели: {:.3f}".format(geo_data_2_rmse))

       init_product  predicted_product
71751        61.212             93.600
80493        41.850             75.105
2655         57.777             90.067
53233       100.054            105.162
91141       109.897            115.303
Средний запас предсказанного сырья в третьем регионе, тыс.баррелей: 94.965
Средний запас начального сырья в третьем регионе, тыс.баррелей: 94.884
RMSE модели: 40.030


In [15]:
# также сравним суммарные запасы сырья исходные и предсказанные

print("Суммарный запас начального сырья в первом регионе, тыс.баррелей: {:.3f}".format(geo_data_0_predicted['init_product'].sum()))
print("Суммарный запас предсказанного сырья в первом регионе, тыс.баррелей: {:.3f}\n".format(geo_data_0_predicted['predicted_product'].sum()))

print("Суммарный запас начального сырья во втором регионе, тыс.баррелей: {:.3f}".format(geo_data_1_predicted['init_product'].sum()))
print("Суммарный запас предсказанного сырья во втором регионе, тыс.баррелей: {:.3f}\n".format(geo_data_1_predicted['predicted_product'].sum()))

print("Суммарный запас начального сырья в третьем регионе, тыс.баррелей: {:.3f}".format(geo_data_2_predicted['init_product'].sum()))
print("Суммарный запас предсказанного сырья в третьем регионе, тыс.баррелей: {:.3f}".format(geo_data_2_predicted['predicted_product'].sum()))

Суммарный запас начального сырья в первом регионе, тыс.баррелей: 2301964.919
Суммарный запас предсказанного сырья в первом регионе, тыс.баррелей: 2314814.195

Суммарный запас начального сырья во втором регионе, тыс.баррелей: 1718078.401
Суммарный запас предсказанного сырья во втором регионе, тыс.баррелей: 1718213.672

Суммарный запас начального сырья в третьем регионе, тыс.баррелей: 2372105.820
Суммарный запас предсказанного сырья в третьем регионе, тыс.баррелей: 2374126.149


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Как можно заметить по данным, приведенным выше, предсказания по регионам получаются достаточно точными. Что касается показателя средний предсказанный запас сырья, то он наибольший в регионе №3, затем идет регион под №1 и №2. А вот показатель RMSE наименьший в регионе №2, причем значение аномально низкое, похоже на то, что данные были выведены исскусственно либо модель переобучилась, затем идет показатель RMSE региона №1 и №3. Далее перейдем к подготовке данных к расчету прибыли.
</div>

## Подготовка к расчёту прибыли

Подготовьтесь к расчёту прибыли:
3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
3.3. Напишите выводы по этапу подготовки расчёта прибыли.

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Для начала создадим отдельные переменные для ключевых значений экономических показателей. Т.к. у нас по условию доход с каждой единицы продукта составляет 450 тыс. рублей и объём указан в тысячах баррелей, бюджет в 10 млрд рублей представим как 10 млн рублей, таким образом сократив значения на тысячу.
</div>

In [16]:
# вводим переменные для экономического расчета
budget = 1e+07
reseacrh_points = 500
selection_points = 200
price_per_barrel = 450

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к расчету достаточного объёма сырья для безубыточной разработки новой скважины и сравним полученный объём сырья со средним запасом в каждом регионе.
</div>

In [17]:
#для начала рассчитаем затраты на одну отобранную скважину

profit_per_point = budget / selection_points
print("Затраты на одну отобранную скважину, тыс.руб.: {:.0f}".format(profit_per_point))

#далее рассчитаем объем нефти на новой скважине для безубыточной разработки

barrels_per_point = profit_per_point / price_per_barrel
print("Объем нефти на новой скважине для безубыточной разработки, тыс.баррелей: {:.0f}".format(barrels_per_point))

#также рассчитаем объем нефти в регионе для безубыточной разработки

barrels_region = budget / price_per_barrel
print("Объем нефти в регионе для безубыточной разработки, тыс.баррелей: {:.0f}".format(barrels_region))

Затраты на одну отобранную скважину, тыс.руб.: 50000
Объем нефти на новой скважине для безубыточной разработки, тыс.баррелей: 111
Объем нефти в регионе для безубыточной разработки, тыс.баррелей: 22222


In [18]:
#далее посмотрим какое количество фактического сырья приходится в среднем на скважину в каждом регионе

print("Средний запас начального сырья в первом регионе на одну скважину, тыс.баррелей: {:.3f}".
      format(geo_data_0_predicted['init_product'].mean()))

print("Средний запас начального сырья во втором регионе на одну скважину, тыс.баррелей: {:.3f}".
      format(geo_data_1_predicted['init_product'].mean()))

print("Средний запас начального сырья в третьем регионе на одну скважину, тыс.баррелей: {:.3f}".
      format(geo_data_2_predicted['init_product'].mean()))

Средний запас начального сырья в первом регионе на одну скважину, тыс.баррелей: 92.079
Средний запас начального сырья во втором регионе на одну скважину, тыс.баррелей: 68.723
Средний запас начального сырья в третьем регионе на одну скважину, тыс.баррелей: 94.884


In [19]:
#также напишем функцию для расчета прибыли на 500 исследуемых точках

def profit_calc(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return price_per_barrel * selected.sum() - budget

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Мы получили достаточно высокое значение объема нефти на новой скважине для безубыточной добычи сырья - 111 тыс. баррелей. Однако если смотреть на наши средние значения запасов начального сырья в каждом регионе, то оно меньше полученного значения. Но опять же у нас были получены средние значения, то есть в регионе есть скважины, запасы по которым как больше, так и меньше среднего значения. Больше всего запасов на одну скважину приходится в тртеьем регионе, затем идет первый регион. Во втором регионе запасов на одну скважину явно меньше.
</div>

## Расчёт прибыли и рисков 

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Для начала напишем функцию для проведения бутстрепа на 3 регионах.
</div>

In [20]:
state = np.random.RandomState(12345)

def bootstrap(target, probs):
    values=[]
    lost = 0
    for i in range(1000):
        target_subsample = target.sample(n=reseacrh_points, replace=True, random_state=state)
        probs_subsample = probs[target_subsample.index]
        profit = profit_calc(target, probs_subsample, selection_points)
        if profit < 0:
            lost +=1
        values.append(profit)
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = lost / 1000
    return mean, lower, upper, risk

In [21]:
#рассчитаем прибыль и риски по первому региону

mean_0, lower_0, upper_0, risk_0 = bootstrap(
                                                geo_data_0_predicted['init_product'], 
                                                geo_data_0_predicted['predicted_product'])

print('Средняя прибыль по первому региону, тыс.руб.: {:.2f}'.format(mean_0))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_0, upper_0))
print('Процент риска: {:.2%}'.format(risk_0))

Средняя прибыль по первому региону, тыс.руб.: 396164.98
95% доверительный интервал от -111215.55 до 909766.94 тыс.руб.
Процент риска: 6.90%


In [22]:
#рассчитаем прибыль и риски по второму региону

mean_1, lower_1, upper_1, risk_1 = bootstrap(
                                                geo_data_1_predicted['init_product'], 
                                                geo_data_1_predicted['predicted_product'])

print('Средняя прибыль по второму региону, тыс.руб.: {:.2f}'.format(mean_1))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_1, upper_1))
print('Процент риска: {:.2%}'.format(risk_1))

Средняя прибыль по второму региону, тыс.руб.: 461155.82
95% доверительный интервал от 78050.81 до 862952.06 тыс.руб.
Процент риска: 0.70%


In [23]:
#рассчитаем прибыль и риски по третьему региону

mean_2, lower_2, upper_2, risk_2 = bootstrap(
                                                geo_data_2_predicted['init_product'], 
                                                geo_data_2_predicted['predicted_product'])

print('Средняя прибыль по третьему региону, тыс.руб.: {:.2f}'.format(mean_2))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_2, upper_2))
print('Процент риска: {:.2%}'.format(risk_2))

Средняя прибыль по третьему региону, тыс.руб.: 392950.48
95% доверительный интервал от -112227.63 до 934562.91 тыс.руб.
Процент риска: 6.50%


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
После проведения всех расчетов нам удалось выяснить, что по вероятности убытков для дальнейшей разработки подходит только регион под номером 2. Вероятность убытков от неудачной разработки регионов 1 и 3 достаточно велика. Также 95%-ный доверительный интервал дает нам понять, что в любом случае при разработке скважин второго региона мы получаем прибыль.
</div>